In [ ]:
# PROCESADO DE DATOS
import numpy as np
import pandas as pd
# VISUALIZACION
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import ijson
import json
import random
# UNDER SAMPLING
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

import missingno as msno
from sklearn.decomposition import PCA

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [ ]:
"""
reviews = []
with open('data/yelp_academic_dataset_review.json') as fl:
    i=0
    for review in fl:
        reviews.append(json.loads(review))
        i+=1
        if i + 1 > 10000:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())
"""

df_review = pd.read_csv('data/yelp_reviews.csv')
print(df_review)

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [ ]:
business = []
with open('data/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())



In [ ]:
useful_reviews = len(df_review[df_review["useful"]>0])
cool_reviews = len(df_review[df_review["cool"]>0])
funny_reviews = len(df_review[df_review["funny"]>0])
negative_reviws = len(df_review[df_review["stars"]<2])
positive_reviews =len(df_review[df_review["stars"]>3])
total_reviews = len(df_review)

print("Total reviews: {}".format(total_reviews))
print("Useful reviews: {}".format(useful_reviews))
print("Funny reviews: {}".format(funny_reviews))
print("Cool reviews: {}".format(cool_reviews))
print("Total negative reviews: {}".format(negative_reviws))
print("Total positive reviews: {}".format(positive_reviews))

ELIMINAMOS LAS COLUMNAS INNECESARIAS

In [ ]:
df_review = df_review.drop(["review_id", "user_id", "useful", "funny", "cool", "date"], axis=1)
df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

COMPROBAMOS

In [ ]:
print(df_review.head())
print(df_business.head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

vectorizer = TfidfVectorizer()
lemma = WordNetLemmatizer()

X = []
for r in df_review["text"]: #Por cada review
    txt = word_tokenize(r.lower()) #Separa la review en palabras
    newtxt = ""
    for w in txt: #Por cada palabra en txt
        l = lemma.lemmatize(w) #se hace lo de quitar mayusculas y quitar raice y eso
        newtxt += l+" "
    X.append(newtxt)

X = vectorizer.fit_transform(X)
print(X)

In [ ]:
X= X.toarray()
X=pd.DataFrame(X)
print(X)

In [ ]:
X.describe()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Seaborn Plot Styling
sns.set(style="white", palette="husl")
sns.set_context("poster")
sns.set_style("ticks")

%matplotlib inline

# New imports, specific to this example
from scipy.cluster.hierarchy import dendrogram, linkage

def mai():
  D = 2                     # Dimensionality
  s = 4                     # Separation so we can control how far apart the means are
  mu1 = np.array([0, 0])
  mu2 = np.array([s, s])
  mu3 = np.array([0, s])

  # Call linkage on X, pass in parameter telling what type of linkage to use
  Z = linkage(X, 'ward')
  # In general, Z will have format index1, index2 for two index's in X that represent the
  # points that are joined at that moment. The third column will be the distance, which is
  # how far apart the two clusters were before being joined, and the 4th column will be
  # sample count, which is the number of points in that cluster. Hence, the size of Z will
  # be ((N - 1) x 4)
  print("Z shape", Z.shape)

  fig, ax = plt.subplots(figsize=(12,8))
  plt.title("Ward")
  dendrogram(Z)      # Call dendrogram on Z
  plt.show()

  # Now do single linkage for a different example
  fig, ax = plt.subplots(figsize=(12,8))
  Z = linkage(X, 'single')
  plt.title("Single")
  dendrogram(Z)
  plt.show()

  # And one more for complete
  fig, ax = plt.subplots(figsize=(12,8))
  Z = linkage(X, 'complete')
  plt.title("Complete")
  dendrogram(Z)
  plt.show()


In [ ]:
mai()

In [ ]:
class Distance_computation_grid(object):
    '''
        class to enable the Computation of distance matrix
    '''
    def __init__(self):
        pass

    def compute_distance(self,samples):
        '''
            Creates a matrix of distances between individual samples and clusters attained at a particular step
        '''
        Distance_mat = np.zeros((len(samples),len(samples)))
        for i in range(Distance_mat.shape[0]):
            for j in range(Distance_mat.shape[0]):
                if i!=j:
                    Distance_mat[i,j] = float(self.distance_calculate(samples[i],samples[j]))
                else:
                    Distance_mat[i,j] = 10**4
        return Distance_mat


    def distance_calculate(self,sample1,sample2):
        '''
            Distance calulated between two samples. The two samples can be both samples, both clusters or
            one cluster and one sample. If both of them are samples/clusters, then simple norm is used. In other
            cases, we refer it as an exception case and pass the samples as parameter to some function that
            calculates the necessary distance between cluster and a sample
        '''
        dist = []
        for i in range(len(sample1)):
            for j in range(len(sample2)):
                try:
                    dist.append(np.linalg.norm(np.array(sample1[i])-np.array(sample2[j])))
                except:
                    dist.append(self.intersampledist(sample1[i],sample2[j]))
        return min(dist)


    def intersampledist(self,s1,s2):
        '''
            To be used in case we have one sample and one cluster . It takes the help of one
            method 'interclusterdist' to compute the distances between elements of a cluster(which are
            samples) and the actual sample given.
        '''
        if str(type(s2[0]))!='<class \'list\'>':
            s2=[s2]
        if str(type(s1[0]))!='<class \'list\'>':
            s1=[s1]
        m = len(s1)
        n = len(s2)
        dist = []
        if n>=m:
            for i in range(n):
                for j in range(m):
                    if (len(s2[i])>=len(s1[j])) and str(type(s2[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s2[i],s1[j]))
                    else:
                        dist.append(np.linalg.norm(np.array(s2[i])-np.array(s1[j])))
        else:
            for i in range(m):
                for j in range(n):
                    if (len(s1[i])>=len(s2[j])) and str(type(s1[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s1[i],s2[j]))
                    else:
                        dist.append(np.linalg.norm(np.array(s1[i])-np.array(s2[j])))
        return min(dist)

    def interclusterdist(self,cl,sample):
        if sample[0]!='<class \'list\'>':
            sample = [sample]
        dist   = []
        for i in range(len(cl)):
            for j in range(len(sample)):
                dist.append(np.linalg.norm(np.array(cl[i])-np.array(sample[j])))
        return min(dist)

In [ ]:
progression = [[i] for i in range(X.shape[0])]
samples     = [[list(X[i])] for i in range(X.shape[0])]
m = len(samples)
distcal  = Distance_computation_grid()

while m>1:
    print('Sample size before clustering    :- ',m)
    Distance_mat      = distcal.compute_distance(samples)
    sample_ind_needed = np.where(Distance_mat==Distance_mat.min())[0]
    value_to_add      = samples.pop(sample_ind_needed[1])
    samples[sample_ind_needed[0]].append(value_to_add)

    print('Cluster Node 1                   :-',progression[sample_ind_needed[0]])
    print('Cluster Node 2                   :-',progression[sample_ind_needed[1]])

    progression[sample_ind_needed[0]].append(progression[sample_ind_needed[1]])
    progression[sample_ind_needed[0]] = [progression[sample_ind_needed[0]]]
    v = progression.pop(sample_ind_needed[1])
    m = len(samples)

    print('Progression(Current Sample)      :-',progression)
    print('Cluster attained                 :-',progression[sample_ind_needed[0]])
    print('Sample size after clustering     :-',m)
    print('\n')

In [ ]:
# Import the fcluster and linkage functions
from scipy.cluster.hierarchy import fcluster, linkage

# Use the linkage() function
distance_matrix = linkage(X, method = 'ward', metric = 'euclidean')

In [ ]:
# Import the dendrogram function
from scipy.cluster.hierarchy import dendrogram

# Create a dendrogram
dn = dendrogram(distance_matrix)

# Display the dendogram
plt.show()